In [118]:
import pymysql
import numpy as np
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd

In [119]:
# connection
mydb = pymysql.connect(host="localhost", user="root", password="", db="sqdb")
query = mydb.cursor()

In [120]:
# users data
Organization_id = '1014'
employee_id = '2012110001'
from_date = '2016-12-28'
to_date = '2021-01-30'
type_id = '10'

# Sales

In [121]:
# salesAmountFromTo
query.execute("""
Select Sum(b.Sales_price) as Sales_price, a.Agent_id From product_sales_agent_master a
INNER JOIN product_sales_agent_detail b ON(b.sales_id = a.sales_id)
INNER JOIN agent_assign_to_marketing_executive c ON(c.Agent_id  = a.Agent_id )
Where a.Organization_id = """+Organization_id+""" and c.employee_id = """+employee_id+"""
and c.end_date is NULL and a.Sales_date Between '"""+from_date+"""' and '"""+to_date+"""'  
and a.Transaction_posting_type = 2 and a.Item_Type = """+type_id+"""
Group by a.Agent_id
""")
salesAmountFromTo = query.fetchall()

# salesAmountUpTo
query.execute("""
Select Sum(b.Sales_price) as Sales_price, a.Agent_id From product_sales_agent_master a 
INNER JOIN product_sales_agent_detail b ON(b.sales_id = a.sales_id)
INNER JOIN agent_assign_to_marketing_executive c ON(c.Agent_id  = a.Agent_id )
Where a.Organization_id = """+Organization_id+""" and c.employee_id = """+employee_id+"""
and a.Sales_date < '"""+from_date+"""'
and c.end_date is NULL and a.Transaction_posting_type = 2
and a.Item_Type = """+type_id+"""
Group by a.Agent_id
""")
salesAmountUpTo = query.fetchall()

df_salesAmountFromTo = pd.DataFrame( [[j for j in i] for i in salesAmountFromTo] )
df_salesAmountFromTo.rename(columns={0: 'Sales_price', 1: 'Agent_id'}, inplace=True);
# df_salesAmountFromTo.head()

df_salesAmountUpTo = pd.DataFrame( [[j for j in i] for i in salesAmountUpTo] )
df_salesAmountUpTo.rename(columns={0: 'Sales_price_upto', 1: 'Agent_id'}, inplace=True);
df_salesAmountUpTo.head()

,Sales_price_upto,Agent_id
0,1562.00,10140001
1,312901.04,10140002
2,21881.00,10140003
3,26549.28,10140004
4,428791.10,10140005


# Collection 

In [122]:
# collection_from_to
query.execute("""
Select Sum(b.Pay_amount) as Pay_amount, a.Agent_id
From payment_collection_agent_master a 
INNER JOIN payment_collection_agent_detail b ON(b.Payment_id = a.Payment_id)
INNER JOIN agent_assign_to_marketing_executive c ON(c.Agent_id = a.Agent_id)
Where a.Organization_id = """+Organization_id+""" and c.employee_id = """+employee_id+"""
and c.end_date is NULL and a.Payment_date Between '"""+from_date+"""' and '"""+to_date+"""'
and a.Type_id = """+type_id+"""
Group by a.Agent_id
""")
collection_from_to = query.fetchall()

# salesReturnAmountUpTo
query.execute("""
Select Sum(b.Pay_amount) as Pay_amount, a.Agent_id
From payment_collection_agent_master a 
INNER JOIN payment_collection_agent_detail b ON(b.Payment_id = a.Payment_id)
INNER JOIN agent_assign_to_marketing_executive c ON(c.Agent_id = a.Agent_id)
Where a.Organization_id = """+Organization_id+""" and c.employee_id = """+employee_id+""" and c.end_date is NULL
and a.Payment_date < '"""+from_date+"""'
and a.Type_id = """+type_id+"""
Group by a.Agent_id
""")
collection_upto = query.fetchall()

df_collection_from_to = pd.DataFrame( [[j for j in i] for i in collection_from_to] )
df_collection_from_to.rename(columns={0: 'Pay_amount', 1: 'Agent_id'}, inplace=True);
# df_collection_from_to.head()

df_collection_upto = pd.DataFrame( [[j for j in i] for i in collection_upto] )
df_collection_upto.rename(columns={0: 'Pay_amount_upto', 1: 'Agent_id'}, inplace=True);
df_collection_upto.head()

,Pay_amount_upto,Agent_id
0,1562.00,10140001
1,102000.00,10140002
2,5000.00,10140003
3,21172.00,10140004
4,243900.00,10140005


# Return

In [123]:
# ReturnAmountFromTo
query.execute("""
Select Sum(b.Return_price) as Return_price, c.Agent_id From product_sales_agent_return_master a 
INNER JOIN product_sales_agent_return_detail b ON(b.Sales_return_id = a.Sales_return_id)
INNER JOIN product_sales_agent_master c ON(c.sales_id = a.sales_id)
INNER JOIN agent_assign_to_marketing_executive d ON(d.Agent_id = c.Agent_id)
Where c.Organization_id = """+Organization_id+""" and d.end_date is NULL and d.employee_id = """+employee_id+"""
and a.Return_date Between '"""+from_date+"""' and '"""+to_date+"""'  
and a.Transaction_type = 2 and c.Item_Type = """+type_id+"""
Group by c.Agent_id
""")
ReturnAmountFromTo = query.fetchall()

# ReturnAmountUpTo
query.execute("""
Select Sum(b.Return_price) as Return_price, c.Agent_id
From product_sales_agent_return_master a 
INNER JOIN product_sales_agent_return_detail b ON(b.Sales_return_id = a.Sales_return_id)
INNER JOIN product_sales_agent_master c ON(c.sales_id = a.sales_id)
INNER JOIN agent_assign_to_marketing_executive d ON(d.Agent_id = c.Agent_id)
Where c.Organization_id = """+Organization_id+""" and d.end_date is NULL and d.employee_id = """+employee_id+"""
and a.Return_date < '"""+from_date+"""' and a.Transaction_type = 2 and c.Item_Type  = """+type_id+"""
Group by c.Agent_id
""")
ReturnAmountUpTo = query.fetchall()

df_ReturnAmountFromTo = pd.DataFrame( [[j for j in i] for i in ReturnAmountFromTo] )
df_ReturnAmountFromTo.rename(columns={0: 'Return_price', 1: 'Agent_id'}, inplace=True);
# df_ReturnAmountFromTo.head()

df_ReturnAmountUpTo = pd.DataFrame( [[j for j in i] for i in ReturnAmountUpTo] )
df_ReturnAmountUpTo.rename(columns={0: 'Return_price_upto', 1: 'Agent_id'}, inplace=True);
df_ReturnAmountUpTo.head()

,Return_price_upto,Agent_id
0,5572.70,10140018
1,3414.40,10140022
2,24475.44,10140023
3,73423.21,10140026
4,23818.08,10140038


In [124]:
query.execute("""
Select a.Agent_id, a.Agent_Name, a.Address1
From sales_agent_information a 
INNER JOIN agent_assign_to_marketing_executive b ON(a.Agent_id = b.Agent_id)
INNER JOIN employee_information c ON(b.employee_id  = c.employee_id )
Where a.Organization_id = """+Organization_id+""" and b.employee_id = """+employee_id+"""
AND b.status = 1
GROUP BY a.Agent_id
""")
AgentList = query.fetchall()
df_agent = pd.DataFrame( [[j for j in i] for i in AgentList] )
df_agent.rename(columns={0: 'Agent_id', 1: 'Agent_name', 2: 'Agent_address'}, inplace=True);
df_agent.head()

,Agent_id,Agent_name,Agent_address
0,10140001,Al-Bakara Electric,"Marina Nodi Bangla Complex, Bogra"
1,10140002,Ali Electric Fare,"Dokhin Bus Stand,Gobindogonj"
2,10140003,Antara Electric,"Registry office more, Sherpur, Bogra"
3,10140004,Asia Electric,"Khan Market, Bogra"
4,10140005,Biddut Bitan,"Mukundogati Bazar, Belkuchi, Shirajgonj"


# Concate with each files

In [126]:
# Files are
# df_salesAmountFromTo, df_salesAmountUpTo
# df_collection_from_to, df_collection_upto
# df_ReturnAmountFromTo, df_ReturnAmountUpTo

In [127]:
df1 = pd.merge(df_agent, df_salesAmountFromTo, how="left", on=["Agent_id", "Agent_id"]).fillna(0)
df1 = pd.merge(df1, df_salesAmountUpTo, how="left", on=["Agent_id", "Agent_id"]).fillna(0)
df1 = pd.merge(df1, df_collection_from_to, how="left", on=["Agent_id", "Agent_id"]).fillna(0)
df1 = pd.merge(df1, df_collection_upto, how="left", on=["Agent_id", "Agent_id"]).fillna(0)
df1 = pd.merge(df1, df_ReturnAmountFromTo, how="left", on=["Agent_id", "Agent_id"]).fillna(0)
df1 = pd.merge(df1, df_ReturnAmountUpTo, how="left", on=["Agent_id", "Agent_id"]).fillna(0)
df1.head()

,Agent_id,Agent_name,Agent_address,Sales_price,Sales_price_upto,Pay_amount,Pay_amount_upto,Return_price,Return_price_upto
0,10140001,Al-Bakara Electric,"Marina Nodi Bangla Complex, Bogra",0,1562.00,0,1562.00,0,0
1,10140002,Ali Electric Fare,"Dokhin Bus Stand,Gobindogonj",25992.00,312901.04,0,102000.00,0,0
2,10140003,Antara Electric,"Registry office more, Sherpur, Bogra",165503.40,21881.00,135000.00,5000.00,12939.00,0
3,10140004,Asia Electric,"Khan Market, Bogra",24941.02,26549.28,9450.00,21172.00,0,0
4,10140005,Biddut Bitan,"Mukundogati Bazar, Belkuchi, Shirajgonj",0,428791.10,167739.00,243900.00,0,0


In [128]:
df1['Outstanding'] = df1['Sales_price_upto'] - df1['Return_price_upto'] - df1['Pay_amount_upto']
df1['Current_Outstanding'] = df1['Outstanding'] + df1['Sales_price'] - df1['Return_price'] - df1['Pay_amount']
df = df1[['Agent_name', 'Agent_address', 'Outstanding', 'Sales_price', 'Return_price', 'Pay_amount', 'Current_Outstanding']]
df

,Agent_name,Agent_address,Outstanding,Sales_price,Return_price,Pay_amount,Current_Outstanding
0,Al-Bakara Electric,"Marina Nodi Bangla Complex, Bogra",0.00,0,0,0,0.00
1,Ali Electric Fare,"Dokhin Bus Stand,Gobindogonj",210901.04,25992.00,0,0,236893.04
2,Antara Electric,"Registry office more, Sherpur, Bogra",16881.00,165503.40,12939.00,135000.00,34445.40
3,Asia Electric,"Khan Market, Bogra",5377.28,24941.02,0,9450.00,20868.30
4,Biddut Bitan,"Mukundogati Bazar, Belkuchi, Shirajgonj",184891.10,0,0,167739.00,17152.10
...,...,...,...,...,...,...,...
248,Forhad Network,"Nobabarie Road ,Ailtap Ali Market,Bogura.",0,6288.00,0,6288.00,0.00
249,Sonet Electric & Machineries,"Rahman Nogur,Bogura.",0,222205.31,0,150969.06,71236.25
250,Sarker Corporation,"Kamrul Tower New Market, Bogura.",0,19709.76,0,0,19709.76
251,Milon Electric Store,"Khan Market, Bogura Sadar, Bogura",0,0,0,0,0


In [129]:
# Total Outstanding, Sales, Return, Collection, Current outstanding
print('Outstanding:', df['Outstanding'].sum())
print('Total Sales Amount:', df['Sales_price'].sum())
print('Total Return Amount:', df['Return_price'].sum())
print('Total collection Amount:', df['Pay_amount'].sum())
print('Current Amount:', df['Current_Outstanding'].sum())

Outstanding: 7058860.42
Total Sales Amount: 33474707.24
Total Return Amount: 1520439.16
Total collection Amount: 22509467.25
Current Amount: 16503661.25


In [117]:
# df.to_csv('Sales Center Executive Payment Report.csv', encoding='utf-8', index=False)
# df.to_excel(r'Sales Center Executive Payment Report.xlsx', index = False)